In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
db_catboost = DFDB('../trial/catboost.pkl', auto_commit=False)
df_trial_catboost = db_catboost.select()
df_trial_catboost['kfold'] = df_trial_catboost['param'].apply(lambda x: x['kfold'])

In [9]:
catboost666 = ['abs_q01_4',
 'MA_1000MA_std_mean_7',
 'spkt_welch_densitycoeff_2',
 'abs_q25_5',
 'abs_q95_2',
 'abs_max_roll_mean_1000',
 'abs_q95_6',
 'q25_roll_std_100',
 "number_peaks{'n': 10}",
 'abs_q75_2',
 'q05_roll_std_1000',
 'abs_q75_6',
 "value_count{'value': 1}",
 'FFT_Mag_75q0',
 'abs_q75_7',
 'spkt_welch_density__coeff_3',
 'q01_roll_std_100',
 "number_crossing_m{'m': 1}",
 'q05_2',
 "autocorrelation{'lag': 5}"]

In [10]:
catboost342 = ['spkt_welch_density__coeff_3',
 'abs_q25_5',
 'q25_roll_std_100',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 'abs_q01_4',
 'iqr_6',
 'q05_roll_std_100',
 'q05_roll_std_1000',
 'median__roll_std',
 'abs_q01_5',
 "number_peaks{'n': 10}",
 'FFT_Mag_75q0',
 "value_count{'value': 1}",
 'q01_roll_std_100',
 'abs_q95_2',
 'abs_q95_6',
 'MA_1000MA_std_mean_7',
 'q05_roll_std_10',
 'q01_roll_std_1000',
 'abs_max_roll_mean_1000',
 'abs_q75_2',
 'abs_q05_6',
 '5000std_quantile25',
 "number_crossing_m{'m': 1}",
 "autocorrelation{'lag': 5}",
 'q75_roll_std_10',
 'q05_2',
 '5000smoothness_quantile05']

In [11]:
df_kkk = df_train[catboost342]

In [12]:
X = df_kkk.values
X = StandardScaler().fit_transform(X)
df_norm = pd.DataFrame(X, columns=catboost342)
df_corr = df_norm.corr()

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
# CorrMtx(df_norm, dropDuplicates = False)

In [81]:
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# plt.figure(figsize=(15, 15))
# sns.heatmap(df_corr, 
#         xticklabels=df_corr.columns,
#         yticklabels=df_corr.columns, cmap=cmap)

In [28]:
def blacklist_merge(df, columns=None, base_correlation_coefficient=.9):
    
    if type(columns)==type(None):
        columns = df.columns.tolist()
    bcc_ = base_correlation_coefficient
    X = df_train[columns].values
    X = StandardScaler().fit_transform(X)
    df_norm = pd.DataFrame(X, columns=columns)
    df_corr = df_norm.corr()
    
    black_lst = []
    group = {}
    for col in columns:
        if col in black_lst:
            continue
        group[col] = list(df_corr[(df_corr[col]>=bcc_)|(df_corr[col]<=-bcc_)].index)
        black_lst +=  group[col]
    return group

In [75]:
def bubble_merge(df, columns=None, base_correlation_coefficient=.9, coverage_rate=.9):
    
    def is_similar(group1, group2):
        assert type(group1)==list, 'group1 should be a list'
        assert type(group2)==list, 'group2 should be a list'
        total_units = group1 + group2
        unique_units = list(set(total_units))
        common_parts = [col for col in unique_units if total_units.count(col)==2]
        if (len(common_parts)/len(group1) >= coverage_rate) | (len(common_parts)/len(group2) >= coverage_rate):
            return True
        else:
            return False
    
    def merge_group(original_group):
        group = original_group.copy()
        merged_group = group
        dict_list_ = list(group.items())
        is_merged = False
        
        index1 = 1
        for k1, v1 in dict_list_[:-1]:
            for k2,v2 in dict_list_[index1:]:
                    if is_similar(v1, v2):
                        group[k1] = list(set(v1 + v2))
                        del group[k2]
                        merged_group = merge_group(group)
                        is_merged = True
                        break
            if is_merged:
                break
            index1 += 1
        return merged_group
    
    if type(columns)==type(None):
        columns = df.columns.tolist()
    bcc_ = base_correlation_coefficient
    X = df_train[columns].values
    X = StandardScaler().fit_transform(X)
    df_norm = pd.DataFrame(X, columns=columns)
    df_corr = df_norm.corr()

    group = {}
    for col in columns:
        group[col] = list(df_corr[(df_corr[col]>=bcc_)|(df_corr[col]<=-bcc_)].index)
    
    return merge_group(group)

In [62]:
group1 = cc_group(df_norm)
for k,v in group.items():
    print(k, len(v))

spkt_welch_density__coeff_3 4
abs_q25_5 17
number_peaks{'n': 10} 4
value_count{'value': 1} 2
abs_q95_2 6
autocorrelation{'lag': 5} 1
5000smoothness_quantile05 7


In [79]:
group1['abs_q95_2']

['abs_q95_2',
 'abs_q95_6',
 'MA_1000MA_std_mean_7',
 'abs_q75_2',
 'q75_roll_std_10',
 'q05_2']

In [80]:
# columns_ = group1['abs_q95_2']+[col for col in catboost342 if col not in group1['abs_q95_2']]
# df_corr = df_norm[columns_].corr()
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# plt.figure(figsize=(15, 15))
# sns.heatmap(df_corr,  xticklabels=df_corr.columns, yticklabels=df_corr.columns, cmap=cmap, vmin=-1, vmax=1, annot=True, fmt='.1g')

In [57]:
group2 = cc_group2(df_norm)
for k,v in group2.items():
    print(k, len(v))

spkt_welch_density__coeff_3 4
abs_q25_5 19
abs_q75_6 15
value_count{'value': 1} 2
autocorrelation{'lag': 5} 1


In [77]:
group2['abs_q75_6']

['abs_q75_7',
 'abs_q25_5',
 'q05_2',
 'q75_roll_std_10',
 'abs_q95_6',
 'abs_q95_2',
 'iqr_6',
 'abs_q01_5',
 'abs_q75_6',
 'abs_q75_2',
 'median__roll_std',
 'MA_1000MA_std_mean_7',
 'abs_q01_4',
 'abs_q05_6',
 'q25_roll_std_100']

In [82]:
# columns_ = group2['abs_q75_6']+[col for col in catboost342 if col not in group2['abs_q75_6']]
# df_corr = df_norm[columns_].corr()
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# plt.figure(figsize=(15, 15))
# sns.heatmap(df_corr,  xticklabels=df_corr.columns, yticklabels=df_corr.columns, cmap=cmap, vmin=-1, vmax=1, annot=True, fmt='.1g')

In [8]:
X = df_train[tsfresh_columns].values
X = StandardScaler().fit_transform(X)
df_train_scaled = pd.DataFrame(X, columns=tsfresh_columns)
df_corr = df_train_scaled.corr()

In [76]:
X = df_test[tsfresh_columns].values
X = StandardScaler().fit_transform(X)
df_test_scaled = pd.DataFrame(X, columns=tsfresh_columns)

In [30]:
cocef = .9
grouped_col_lst = []
group = {}
for col in df_corr.columns:
    if col in grouped_col_lst:
        continue
    group[col] = list(df_corr[(df_corr[col]>cocef)|(df_corr[col]<-cocef)].index)
    grouped_col_lst +=  list(df_corr[(df_corr[col]>cocef)|(df_corr[col]<-cocef)].index)
#     print(col, len(group[col]))

spkt_welch_density__coeff_28 30
max_last_50000_1 12
value_count{'value': 0} 8
std_roll_mean_100 132
std_last_50000_9 13
std_last_10000_9 12
Hilbert_mean_8 80
max_last_10000_3 12
max_last_10000_5 8
max_last_10000_6 7
spkt_welch_density__coeff_0 3
4th_peak_psd 1
abs_max_8 13
abs_q05_6 30
abs_q05_8 2
abs_q05_9 1
abs_q25_1 14
abs_q75_0 1
abs_q75_8 10
abs_q25_9 5
abs_q01_7 4
abs_max_9 3
max_last_50000_4 12
abs_q01_8 1
5000variance_smoothness_ 5
spkt_welch_density__coeff_31 27
5000variance_quantile25 16
5000variance_median_ 16
max_first_50000_8 5
5000smoothness_median_ 19
5000smoothness_min_ 4
5000smoothness_peak_peak_amp_ 2
5000smoothness_quantile95 4
5000variance_entropy_ 16
5000std_form_factor_ 26
5000std_quantile05 13
5000std_quantile75 22
5000smoothness_std_ 2
FFT_Mag_max17500 7
max_first_50000_9 2
abs_q95_3 61
q01_roll_std_1000 10
change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'} 2
change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'} 4
change_quan

In [31]:
catboost342_corr = []
for col in catboost342:
    for k,v in group.items():
        if col in v:
            catboost342_corr.append(k)
unique_list  = list(set(catboost342_corr))

In [107]:
df_pca = pd.DataFrame()
for col in unique_list:
    N_ = len(group[col])
    df_pca_i = pd.DataFrame(PCA(n_components=N_).fit_transform(df_train_scaled[group[col]].values), columns=['{}_pca{}'.format(col, i) for i in range(N_)])
    df_pca = pd.concat([df_pca, df_pca_i], axis=1)

In [108]:
df_pca_test = pd.DataFrame()
for col in unique_list:
    N_ = len(group[col])
    df_pca_test_i = pd.DataFrame(PCA(n_components=N_).fit_transform(df_test_scaled[group[col]].values), columns=['{}_pca{}'.format(col, i) for i in range(N_)])
    df_pca_test = pd.concat([df_pca_test, df_pca_test_i], axis=1)

In [109]:
df_train = pd.concat([df_train, df_pca], axis=1)
df_test = pd.concat([df_test, df_pca_test], axis=1)

In [110]:
df_train.shape,df_test.shape

((33426, 2332), (2624, 2330))

In [78]:
# df_pca.to_pickle('../feats/pca.pkl')
# df_pca_test.to_pickle('../feats/pca_test.pkl')

In [44]:
df_trial_catboost.loc[666]['param']

{'columns': ['abs_q01_4',
  'MA_1000MA_std_mean_7',
  'spkt_welch_densitycoeff_2',
  'abs_q25_5',
  'abs_q95_2',
  'abs_max_roll_mean_1000',
  'abs_q95_6',
  'q25_roll_std_100',
  "number_peaks{'n': 10}",
  'abs_q75_2',
  'q05_roll_std_1000',
  'abs_q75_6',
  "value_count{'value': 1}",
  'FFT_Mag_75q0',
  'abs_q75_7',
  'spkt_welch_density__coeff_3',
  'q01_roll_std_100',
  "number_crossing_m{'m': 1}",
  'q05_2',
  "autocorrelation{'lag': 5}"],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'logging_level': 'Silent',
   'num_trees': 929,
   'depth': 5,
   'learning_rate': 0.01571023413441655,
   'l2_leaf_reg': 0.49142786214498846,
   'bagging_temperature': 0.6008299770689625,
   'random_strength': 0.7930166632962454,
   'random_state': 1569},
  'fit': {}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance':

In [51]:
param = {'columns': df_pca.columns.tolist()+catboost342,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'logging_level': 'Silent',
   'num_trees': 929,
   'depth': 5,
   'learning_rate': 0.01571023413441655,
   'l2_leaf_reg': 0.49142786214498846,
   'bagging_temperature': 0.6008299770689625,
   'random_strength': 0.7930166632962454,
   'random_state': 1569},
  'fit': {}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}


In [58]:
df_feature_importances = pd.DataFrame(mytrial)['df_feature_importances'].values[0]

In [65]:
init_sorted_features = df_feature_importances.sort_values(by=['average_model_weight'], ascending=False)['feature'].tolist()

In [66]:
# mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=10, key='average_model_weight', remark='try pca')

In [69]:
param = {'columns': init_sorted_features[:100],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'logging_level': 'Silent',
   'num_trees': 929,
   'depth': 5,
   'learning_rate': 0.01571023413441655,
   'l2_leaf_reg': 0.49142786214498846,
   'bagging_temperature': 0.6008299770689625,
   'random_strength': 0.7930166632962454,
   'random_state': 1569},
  'fit': {}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}

In [71]:
EP.select_features_(df_train, param, mytrial, nfeats_best=20, nfeats_removed_per_try=10, key='average_model_weight', remark='try pca 2')

In [84]:
def astar_select_features_(df_train, param, trial, cv_score, df_test=None, remark=None):
    param_ = copy.deepcopy(param)
    for col in param_['columns']:
        param_ = copy.deepcopy(param)
        param_['columns'] = list(set(param_['columns'])-set([col]))
        df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param_, df_test=df_test, trial=trial, is_output_feature_importance=False, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean < cv_score:
            astar_select_features_(df_train, param_, trial, val_mae_mean, df_test, remark)
    return

In [87]:
# astar_select_features_(df_train, df_trial.loc[9]['param'], mytrial, df_trial.loc[9].val_mae, remark='try pca 3')

In [88]:
df_trial = pd.DataFrame(mytrial)

In [91]:
# df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae'])

In [75]:
df_trial.loc[9]['param']['columns']

['5000variance_median__pca6',
 'spkt_welch_density__coeff_3',
 'abs_q01_4',
 'q25_roll_std_100',
 'q05_roll_std_1000',
 'q05_roll_std_100',
 'abs_max_roll_mean_1000',
 'spkt_welch_densitycoeff_2',
 'abs_q05_6_pca1',
 '5000std_quantile75_pca4',
 'abs_q25_5',
 'median__roll_std',
 "value_count{'value': 1}",
 "range_count{'min': 0, 'max': 1000000000000.0}_pca1",
 'abs_q95_2',
 "number_crossing_m{'m': 1}",
 'abs_q75_7',
 'MA_1000MA_std_mean_7',
 "number_crossing_m{'m': 1}_pca0",
 'q05_2']

In [92]:
# mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[9]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
                'logging_level': 'Silent'
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune pca 3')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)



[I 2019-05-27 05:19:32,152] Finished a trial resulted in value: 0.40709666038368314. Current best value is 0.40709666038368314 with parameters: {'num_trees': 945, 'depth': 8, 'learning_rate': 0.1280737246202017, 'l2_leaf_reg': 0.9442679104043623, 'bagging_temperature': 0.7737157971761918, 'random_strength': 0.5421647344832982, 'random_state': 4238}.
[I 2019-05-27 05:19:41,713] Finished a trial resulted in value: 0.06796967273397836. Current best value is 0.06796967273397836 with parameters: {'num_trees': 802, 'depth': 2, 'learning_rate': 0.08576641165567109, 'l2_leaf_reg': 0.7339674091025965, 'bagging_temperature': 0.7878016194205634, 'random_strength': 0.2760023249376411, 'random_state': 5206}.
[I 2019-05-27 05:19:50,815] Finished a trial resulted in value: 0.13369961218581292. Current best value is 0.06796967273397836 with parameters: {'num_trees': 802, 'depth': 2, 'learning_rate': 0.08576641165567109, 'l2_leaf_reg': 0.7339674091025965, 'bagging_temperature': 0.7878016194205634, 'ran

[I 2019-05-27 05:41:21,907] Finished a trial resulted in value: 0.3937707523063144. Current best value is 0.0194286481771043 with parameters: {'num_trees': 222, 'depth': 2, 'learning_rate': 0.024917419684503887, 'l2_leaf_reg': 0.01929030841113069, 'bagging_temperature': 0.9960688671496106, 'random_strength': 0.01940914471086613, 'random_state': 167}.
[I 2019-05-27 05:41:28,253] Finished a trial resulted in value: 0.026776307011633967. Current best value is 0.0194286481771043 with parameters: {'num_trees': 222, 'depth': 2, 'learning_rate': 0.024917419684503887, 'l2_leaf_reg': 0.01929030841113069, 'bagging_temperature': 0.9960688671496106, 'random_strength': 0.01940914471086613, 'random_state': 167}.
[I 2019-05-27 05:41:51,692] Finished a trial resulted in value: 0.39229368203343135. Current best value is 0.0194286481771043 with parameters: {'num_trees': 222, 'depth': 2, 'learning_rate': 0.024917419684503887, 'l2_leaf_reg': 0.01929030841113069, 'bagging_temperature': 0.9960688671496106, 

[I 2019-05-27 05:49:29,011] Finished a trial resulted in value: 0.23020448929197826. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591, 'random_strength': 0.8052253090166782, 'random_state': 5039}.
[I 2019-05-27 05:49:36,893] Finished a trial resulted in value: 0.15261520253001334. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591, 'random_strength': 0.8052253090166782, 'random_state': 5039}.
[I 2019-05-27 05:49:46,852] Finished a trial resulted in value: 0.26042855058517206. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.741362154005959

[I 2019-05-27 06:00:12,111] Finished a trial resulted in value: 0.03024535440873019. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591, 'random_strength': 0.8052253090166782, 'random_state': 5039}.
[I 2019-05-27 06:00:25,896] Finished a trial resulted in value: 0.2547253859843307. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591, 'random_strength': 0.8052253090166782, 'random_state': 5039}.
[I 2019-05-27 06:00:51,225] Finished a trial resulted in value: 0.29092225755487516. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591

[I 2019-05-27 06:06:47,855] Finished a trial resulted in value: 0.07717828537600246. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591, 'random_strength': 0.8052253090166782, 'random_state': 5039}.
[I 2019-05-27 06:06:54,187] Finished a trial resulted in value: 0.04058574381054023. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.7413621540059591, 'random_strength': 0.8052253090166782, 'random_state': 5039}.
[I 2019-05-27 06:06:58,947] Finished a trial resulted in value: 0.016669553887624207. Current best value is 0.007075005729999574 with parameters: {'num_trees': 249, 'depth': 3, 'learning_rate': 0.010680742027147097, 'l2_leaf_reg': 0.4998177874179952, 'bagging_temperature': 0.74136215400595

In [113]:
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, df_trial.loc[100]['param'], df_test = df_test, trial=mytrial, remark='tune pca 3-2')

In [123]:
df_trial = pd.DataFrame(mytrial)

In [116]:
df_trial[(df_trial['remark']=='tune pca 3-2')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head()

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
254,2019-05-27 07:37:09.062134,20,1.819569,0.000005,1.867549,0.000112,0.04798


In [120]:
param_ = copy.deepcopy(df_trial.loc[254]['param'])
for col in param_['columns']:
    param_ = copy.deepcopy(df_trial.loc[254]['param'])
    param_['columns'] = list(set(param_['columns'])-set([col]))
    df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param_, df_test=df_test, trial=mytrial, is_output_feature_importance=False, remark='tune pca 3-3')
    val_mae_mean = np.mean(df_his.valid)


In [126]:
df_trial.loc[254]['param']['columns']

['5000variance_median__pca6',
 'spkt_welch_density__coeff_3',
 'abs_q01_4',
 'q25_roll_std_100',
 'q05_roll_std_1000',
 'q05_roll_std_100',
 'abs_max_roll_mean_1000',
 'spkt_welch_densitycoeff_2',
 'abs_q05_6_pca1',
 '5000std_quantile75_pca4',
 'abs_q25_5',
 'median__roll_std',
 "value_count{'value': 1}",
 "range_count{'min': 0, 'max': 1000000000000.0}_pca1",
 'abs_q95_2',
 "number_crossing_m{'m': 1}",
 'abs_q75_7',
 'MA_1000MA_std_mean_7',
 "number_crossing_m{'m': 1}_pca0",
 'q05_2']

In [125]:
for idx in [270,267,273]:
    print(set(df_trial.loc[254]['param']['columns'])-set(df_trial.loc[idx]['param']['columns']))

{"number_crossing_m{'m': 1}"}
{"value_count{'value': 1}"}
{"number_crossing_m{'m': 1}_pca0"}


In [124]:
df_trial[(df_trial['remark']=='tune pca 3-3')].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
270,2019-05-27 07:51:36.666257,19,1.819248,2.082721e-06,1.866576,0.000145,0.047328
267,2019-05-27 07:50:58.161737,19,1.818291,2.040645e-06,1.866827,0.000077,0.048536
273,2019-05-27 07:52:12.967147,19,1.820711,2.802645e-06,1.866840,0.000144,0.046129
258,2019-05-27 07:49:06.224833,19,1.819197,9.399828e-06,1.867321,0.000116,0.048124
264,2019-05-27 07:50:20.930936,19,1.820373,2.847668e-06,1.867573,0.000100,0.047201
260,2019-05-27 07:49:30.463497,19,1.819260,2.585628e-07,1.867923,0.000104,0.048663
274,2019-05-27 07:52:25.683212,19,1.820439,8.401753e-06,1.868264,0.000090,0.047825
269,2019-05-27 07:51:22.901780,19,1.821332,3.396489e-06,1.868450,0.000143,0.047119
265,2019-05-27 07:50:32.930135,19,1.821193,1.498784e-06,1.868590,0.000085,0.047397
266,2019-05-27 07:50:45.501967,19,1.819935,4.755503e-06,1.868836,0.000127,0.048901


In [118]:
idx=254
df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_catboost_pca_{}.csv'.format(idx), index=False)

In [100]:
df_test_pred.shape

(33426, 4)

In [127]:
db_catboost = DFDB('../trial/catboost.pkl', auto_commit=False)
df_trial_catboost = db_catboost.select()
df_trial_catboost['kfold'] = df_trial_catboost['param'].apply(lambda x: x['kfold'])

In [128]:
db_xgbm = DFDB('../trial/xgbm.pkl', auto_commit=False)
df_trial_xgbm = db_xgbm.select()
df_trial_xgbm['kfold'] = df_trial_xgbm['param'].apply(lambda x: x['kfold'])

In [129]:
db_lgbm = DFDB('../trial/lgbm.pkl', auto_commit=False)
df_trial_lgbm = db_lgbm.select()
df_trial_lgbm['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])

In [130]:
db_randomforest = DFDB('../trial/randomforest.pkl', auto_commit=False)
df_trial_randomforest = db_randomforest.select()
df_trial_randomforest['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])

In [131]:
db_extratrees = DFDB('../trial/extratrees.pkl', auto_commit=False)
df_trial_extratrees = db_extratrees.select()
df_trial_extratrees['kfold'] = df_trial_extratrees['param'].apply(lambda x: x['kfold'])

In [132]:
db_gradientboosting = DFDB('../trial/gradientboosting.pkl', auto_commit=False)
df_trial_gradientboosting = db_gradientboosting.select()
df_trial_gradientboosting['kfold'] = df_trial_gradientboosting['param'].apply(lambda x: x['kfold'])

In [133]:
db_knn = DFDB('../trial/knn.pkl', auto_commit=False)
df_trial_knn = db_knn.select()
df_trial_knn['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [134]:
db_svr = DFDB('../trial/svr.pkl', auto_commit=False)
df_trial_svr = db_svr.select()
df_trial_svr['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [135]:
db_fm = DFDB('../trial/fm.pkl', auto_commit=False)
df_trial_fm = db_fm.select()
df_trial_fm['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [136]:
db_lasso = DFDB('../trial/lasso.pkl', auto_commit=False)
df_trial_lasso = db_lasso.select()
df_trial_lasso['kfold'] = df_trial_lasso['param'].apply(lambda x: x['kfold'])

In [137]:
db_ridge = DFDB('../trial/ridge.pkl', auto_commit=False)
df_trial_ridge = db_ridge.select()
df_trial_ridge['kfold'] = df_trial_ridge['param'].apply(lambda x: x['kfold'])

In [141]:
db_kerasmlp = DFDB('../trial/kerasmlp.pkl', auto_commit=False)
df_trial_mlp = db_kerasmlp.select()
df_trial_mlp['kfold'] = df_trial_kerasmlp['param'].apply(lambda x: x['kfold'])

In [139]:
db_frgf = DFDB('../trial/frgf.pkl', auto_commit=False)
df_trial_frgf = db_frgf.select()
df_trial_frgf['kfold'] = df_trial_frgf['param'].apply(lambda x: x['kfold'])

In [169]:
#stratified
trial_lst = [
    df_trial_catboost.loc[452], df_trial_xgbm.loc[1172], df_trial_lgbm.loc[2156],df_trial_gradientboosting.loc[306],
             df_trial_randomforest.loc[297],df_trial_extratrees.loc[459],
            df_trial_knn.loc[17], df_trial_svr.loc[7],df_trial_fm.loc[313],
            df_trial_lasso.loc[8], df_trial_ridge.loc[15],
        df_trial_mlp.loc[13], df_trial_frgf.loc[200],
            ]
name_lst = [
    'cb452', 'xgbm1172', 'lgbm2156',
    'gbm306','rf297','et459',
    'knn17','svr7', 'fm313',
    'lasso8','ridge15',
    'mlp13','frgf200'
]

trial_lst2=[
    df_trial_catboost.loc[452], df_trial_xgbm.loc[1172], df_trial_lgbm.loc[2156],df_trial_gradientboosting.loc[306],
             df_trial_randomforest.loc[297],df_trial_extratrees.loc[459],
            df_trial_lasso.loc[8], df_trial_ridge.loc[15],
        df_trial_frgf.loc[200],
            ]

In [172]:
t['param']

{'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'spkt_welch_density__coeff_3',
  'q05_roll_std_100',
  'abs_q25_5',
  'spkt_welch_densitycoeff_2',
  'iqr_6',
  "number_peaks{'n': 10}",
  'q05_roll_std_1000',
  'abs_q75_6',
  'abs_q95_2',
  'median__roll_std',
  'q05_5',
  'abs_q75_7',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'spkt_welch_density__coeff_25',
  'spkt_welch_density__coeff_4',
  'abs_max_1',
  '5000std_quantile05',
  'agg_autocorrelationf_agg_"mean"__maxlag_40',
  'Hilbert_mean_2',
  'FFT_Mag_75q0',
  '5000smoothness_entropy_',
  'MA_1000MA_std_mean_7',
  "number_peaks{'n': 5}"],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'xgb.XGBRegressor',
  'init': {'max_depth': 3,
   'max_bin': 77,
   'eta': 0.28071904276671816,
   'colsample_bytree': 0.9102857670808742,
   'min_child_weight': 271,
   'n_estimators': 113,
   'subsample': 0

In [173]:
trial_comp = []
for t in trial_lst2:
    args = copy.deepcopy(t['param'])
    args['columns'] = args['columns']+pca_columns
    args['scaler']['init']={}
    args['algorithm']['fit']={}
    df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, args, df_test=df_test, trial=trial_comp, is_output_feature_importance=False, remark='try conmp')

0:	learn: 6.6522795	total: 9.55ms	remaining: 8.87s
1:	learn: 6.5653605	total: 20.6ms	remaining: 9.57s
2:	learn: 6.4805513	total: 32.2ms	remaining: 9.94s
3:	learn: 6.3966835	total: 44.6ms	remaining: 10.3s
4:	learn: 6.3141530	total: 56.3ms	remaining: 10.4s
5:	learn: 6.2332967	total: 67.3ms	remaining: 10.4s
6:	learn: 6.1541431	total: 79.4ms	remaining: 10.5s
7:	learn: 6.0762146	total: 91.6ms	remaining: 10.5s
8:	learn: 5.9998390	total: 105ms	remaining: 10.8s
9:	learn: 5.9253070	total: 118ms	remaining: 10.8s
10:	learn: 5.8515843	total: 131ms	remaining: 10.9s
11:	learn: 5.7792057	total: 144ms	remaining: 11s
12:	learn: 5.7087049	total: 157ms	remaining: 11.1s
13:	learn: 5.6392124	total: 169ms	remaining: 11.1s
14:	learn: 5.5711626	total: 181ms	remaining: 11.1s
15:	learn: 5.5042738	total: 193ms	remaining: 11s
16:	learn: 5.4390694	total: 205ms	remaining: 11s
17:	learn: 5.3747747	total: 216ms	remaining: 10.9s
18:	learn: 5.3116374	total: 226ms	remaining: 10.8s
19:	learn: 5.2494578	total: 236ms	remai

161:	learn: 2.5983697	total: 1.87s	remaining: 8.84s
162:	learn: 2.5963453	total: 1.88s	remaining: 8.82s
163:	learn: 2.5943246	total: 1.89s	remaining: 8.81s
164:	learn: 2.5919600	total: 1.9s	remaining: 8.79s
165:	learn: 2.5903329	total: 1.91s	remaining: 8.77s
166:	learn: 2.5885423	total: 1.92s	remaining: 8.76s
167:	learn: 2.5867766	total: 1.93s	remaining: 8.74s
168:	learn: 2.5849476	total: 1.94s	remaining: 8.72s
169:	learn: 2.5836989	total: 1.95s	remaining: 8.71s
170:	learn: 2.5817802	total: 1.96s	remaining: 8.7s
171:	learn: 2.5799932	total: 1.97s	remaining: 8.67s
172:	learn: 2.5781650	total: 1.98s	remaining: 8.65s
173:	learn: 2.5762109	total: 1.99s	remaining: 8.63s
174:	learn: 2.5748429	total: 2s	remaining: 8.62s
175:	learn: 2.5732610	total: 2.01s	remaining: 8.6s
176:	learn: 2.5714945	total: 2.02s	remaining: 8.59s
177:	learn: 2.5698993	total: 2.03s	remaining: 8.58s
178:	learn: 2.5684792	total: 2.04s	remaining: 8.57s
179:	learn: 2.5666335	total: 2.06s	remaining: 8.56s
180:	learn: 2.5648

320:	learn: 2.4699226	total: 3.65s	remaining: 6.91s
321:	learn: 2.4695496	total: 3.66s	remaining: 6.9s
322:	learn: 2.4690225	total: 3.67s	remaining: 6.89s
323:	learn: 2.4684949	total: 3.68s	remaining: 6.88s
324:	learn: 2.4680401	total: 3.69s	remaining: 6.87s
325:	learn: 2.4675891	total: 3.7s	remaining: 6.85s
326:	learn: 2.4669733	total: 3.71s	remaining: 6.83s
327:	learn: 2.4665065	total: 3.72s	remaining: 6.82s
328:	learn: 2.4662111	total: 3.73s	remaining: 6.81s
329:	learn: 2.4658561	total: 3.75s	remaining: 6.8s
330:	learn: 2.4652863	total: 3.76s	remaining: 6.79s
331:	learn: 2.4649078	total: 3.77s	remaining: 6.78s
332:	learn: 2.4643241	total: 3.78s	remaining: 6.77s
333:	learn: 2.4638873	total: 3.79s	remaining: 6.76s
334:	learn: 2.4632806	total: 3.8s	remaining: 6.75s
335:	learn: 2.4630367	total: 3.81s	remaining: 6.73s
336:	learn: 2.4628529	total: 3.82s	remaining: 6.72s
337:	learn: 2.4626287	total: 3.83s	remaining: 6.7s
338:	learn: 2.4621760	total: 3.85s	remaining: 6.69s
339:	learn: 2.461

479:	learn: 2.4200165	total: 5.4s	remaining: 5.05s
480:	learn: 2.4197934	total: 5.41s	remaining: 5.04s
481:	learn: 2.4194454	total: 5.42s	remaining: 5.03s
482:	learn: 2.4192923	total: 5.44s	remaining: 5.02s
483:	learn: 2.4191160	total: 5.45s	remaining: 5.01s
484:	learn: 2.4186103	total: 5.46s	remaining: 5s
485:	learn: 2.4183097	total: 5.47s	remaining: 4.99s
486:	learn: 2.4181516	total: 5.48s	remaining: 4.97s
487:	learn: 2.4180130	total: 5.49s	remaining: 4.96s
488:	learn: 2.4176836	total: 5.5s	remaining: 4.95s
489:	learn: 2.4173483	total: 5.52s	remaining: 4.94s
490:	learn: 2.4171561	total: 5.53s	remaining: 4.93s
491:	learn: 2.4169413	total: 5.54s	remaining: 4.92s
492:	learn: 2.4165536	total: 5.55s	remaining: 4.91s
493:	learn: 2.4163880	total: 5.57s	remaining: 4.9s
494:	learn: 2.4162754	total: 5.58s	remaining: 4.89s
495:	learn: 2.4160997	total: 5.59s	remaining: 4.88s
496:	learn: 2.4157865	total: 5.6s	remaining: 4.87s
497:	learn: 2.4155854	total: 5.61s	remaining: 4.86s
498:	learn: 2.41534

638:	learn: 2.3848066	total: 7.17s	remaining: 3.25s
639:	learn: 2.3846404	total: 7.17s	remaining: 3.24s
640:	learn: 2.3844469	total: 7.19s	remaining: 3.23s
641:	learn: 2.3841114	total: 7.2s	remaining: 3.22s
642:	learn: 2.3839192	total: 7.21s	remaining: 3.21s
643:	learn: 2.3836637	total: 7.22s	remaining: 3.19s
644:	learn: 2.3832533	total: 7.23s	remaining: 3.18s
645:	learn: 2.3830371	total: 7.24s	remaining: 3.17s
646:	learn: 2.3827773	total: 7.25s	remaining: 3.16s
647:	learn: 2.3824855	total: 7.26s	remaining: 3.15s
648:	learn: 2.3822613	total: 7.27s	remaining: 3.14s
649:	learn: 2.3820776	total: 7.28s	remaining: 3.13s
650:	learn: 2.3817959	total: 7.29s	remaining: 3.11s
651:	learn: 2.3814898	total: 7.3s	remaining: 3.1s
652:	learn: 2.3813628	total: 7.32s	remaining: 3.09s
653:	learn: 2.3811459	total: 7.33s	remaining: 3.08s
654:	learn: 2.3810133	total: 7.34s	remaining: 3.07s
655:	learn: 2.3808388	total: 7.35s	remaining: 3.06s
656:	learn: 2.3805424	total: 7.36s	remaining: 3.05s
657:	learn: 2.3

...

373:	learn: 2.4313214	total: 4.21s	remaining: 6.24s
374:	learn: 2.4309833	total: 4.21s	remaining: 6.23s
375:	learn: 2.4305170	total: 4.22s	remaining: 6.21s
376:	learn: 2.4302216	total: 4.24s	remaining: 6.2s
377:	learn: 2.4298143	total: 4.24s	remaining: 6.19s
378:	learn: 2.4294883	total: 4.25s	remaining: 6.17s
379:	learn: 2.4293109	total: 4.26s	remaining: 6.16s
380:	learn: 2.4290334	total: 4.27s	remaining: 6.15s
381:	learn: 2.4288446	total: 4.28s	remaining: 6.13s
382:	learn: 2.4285406	total: 4.29s	remaining: 6.12s
383:	learn: 2.4282078	total: 4.3s	remaining: 6.11s
384:	learn: 2.4280369	total: 4.32s	remaining: 6.1s
385:	learn: 2.4278821	total: 4.33s	remaining: 6.09s
386:	learn: 2.4274804	total: 4.34s	remaining: 6.08s
387:	learn: 2.4272829	total: 4.35s	remaining: 6.07s
388:	learn: 2.4268955	total: 4.36s	remaining: 6.05s
389:	learn: 2.4266257	total: 4.37s	remaining: 6.04s
390:	learn: 2.4263557	total: 4.38s	remaining: 6.03s
391:	learn: 2.4259320	total: 4.39s	remaining: 6.02s
392:	learn: 2.4

532:	learn: 2.3919423	total: 5.95s	remaining: 4.42s
533:	learn: 2.3917224	total: 5.96s	remaining: 4.41s
534:	learn: 2.3914277	total: 5.97s	remaining: 4.4s
535:	learn: 2.3911685	total: 5.99s	remaining: 4.39s
536:	learn: 2.3909437	total: 6s	remaining: 4.38s
537:	learn: 2.3907955	total: 6.01s	remaining: 4.37s
538:	learn: 2.3906046	total: 6.02s	remaining: 4.36s
539:	learn: 2.3904627	total: 6.04s	remaining: 4.35s
540:	learn: 2.3903189	total: 6.05s	remaining: 4.34s
541:	learn: 2.3899355	total: 6.06s	remaining: 4.33s
542:	learn: 2.3896526	total: 6.07s	remaining: 4.32s
543:	learn: 2.3894522	total: 6.08s	remaining: 4.3s
544:	learn: 2.3892807	total: 6.09s	remaining: 4.29s
545:	learn: 2.3891365	total: 6.11s	remaining: 4.28s
546:	learn: 2.3889388	total: 6.12s	remaining: 4.27s
547:	learn: 2.3887438	total: 6.13s	remaining: 4.26s
548:	learn: 2.3885108	total: 6.14s	remaining: 4.25s
549:	learn: 2.3882599	total: 6.15s	remaining: 4.24s
550:	learn: 2.3880128	total: 6.16s	remaining: 4.23s
551:	learn: 2.387

691:	learn: 2.3586887	total: 7.7s	remaining: 2.63s
692:	learn: 2.3584862	total: 7.71s	remaining: 2.62s
693:	learn: 2.3582772	total: 7.71s	remaining: 2.61s
694:	learn: 2.3580002	total: 7.72s	remaining: 2.6s
695:	learn: 2.3577644	total: 7.74s	remaining: 2.59s
696:	learn: 2.3575839	total: 7.75s	remaining: 2.58s
697:	learn: 2.3572860	total: 7.75s	remaining: 2.57s
698:	learn: 2.3571441	total: 7.76s	remaining: 2.56s
699:	learn: 2.3569005	total: 7.78s	remaining: 2.54s
700:	learn: 2.3567118	total: 7.79s	remaining: 2.53s
701:	learn: 2.3564032	total: 7.8s	remaining: 2.52s
702:	learn: 2.3562289	total: 7.81s	remaining: 2.51s
703:	learn: 2.3560570	total: 7.82s	remaining: 2.5s
704:	learn: 2.3559183	total: 7.83s	remaining: 2.49s
705:	learn: 2.3556025	total: 7.85s	remaining: 2.48s
706:	learn: 2.3553852	total: 7.86s	remaining: 2.47s
707:	learn: 2.3551699	total: 7.86s	remaining: 2.45s
708:	learn: 2.3550173	total: 7.88s	remaining: 2.44s
709:	learn: 2.3548453	total: 7.89s	remaining: 2.43s
710:	learn: 2.35

850:	learn: 2.3281989	total: 9.45s	remaining: 866ms
851:	learn: 2.3280658	total: 9.46s	remaining: 855ms
852:	learn: 2.3279389	total: 9.47s	remaining: 844ms
853:	learn: 2.3277904	total: 9.48s	remaining: 833ms
854:	learn: 2.3276544	total: 9.49s	remaining: 822ms
855:	learn: 2.3274935	total: 9.51s	remaining: 811ms
856:	learn: 2.3271989	total: 9.52s	remaining: 800ms
857:	learn: 2.3269906	total: 9.53s	remaining: 789ms
858:	learn: 2.3268091	total: 9.54s	remaining: 778ms
859:	learn: 2.3266537	total: 9.55s	remaining: 766ms
860:	learn: 2.3264962	total: 9.56s	remaining: 755ms
861:	learn: 2.3263206	total: 9.58s	remaining: 744ms
862:	learn: 2.3261260	total: 9.59s	remaining: 733ms
863:	learn: 2.3260250	total: 9.6s	remaining: 722ms
864:	learn: 2.3258537	total: 9.61s	remaining: 711ms
865:	learn: 2.3256679	total: 9.63s	remaining: 700ms
866:	learn: 2.3253975	total: 9.64s	remaining: 689ms
867:	learn: 2.3252515	total: 9.65s	remaining: 678ms
868:	learn: 2.3251568	total: 9.66s	remaining: 667ms
869:	learn: 2

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py:224: UserWarning:

Cannot find FastRGF executable files. FastRGF estimators will be unavailable for usage.



In [179]:
for t in trial_lst2:
    args = copy.deepcopy(t['param'])
    args['columns'] = args['columns']+pca_columns
    args['scaler']['init']={}
    args['algorithm']['fit']={}
    print(args['algorithm']['cls'], t['val_mae'], t['mae_diff'])

cb.CatBoostRegressor 1.9005345457826346 0.04705627937966006
xgb.XGBRegressor 1.9655602276971649 0.040781720685815426
lgb.LGBMRegressor 1.9528967095257748 0.04172569264886117
GradientBoostingRegressor 1.9518365916002054 0.03707554631740417
RandomForestRegressor 2.0862079247944467 0.005860461151117491
ExtraTreesRegressor 2.110346841048832 0.0012003322039348951
Lasso 2.1560981052138293 0.050084509831083146
Ridge 2.180492828109568 0.035332374813775935
RGFRegressor 1.9777535467520957 0.041801952867579484


In [174]:
df_trial_comp = pd.DataFrame(trial_comp)
df_trial_comp['alg-name'] = df_trial_comp['param'].apply(lambda x:x['algorithm']['cls'])

In [175]:
df_trial_comp[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','alg-name']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,alg-name
0,2019-05-27 08:20:31.880167,34,1.837069,0.000014,1.882760,0.000129,0.045691,cb.CatBoostRegressor
1,2019-05-27 08:20:33.919146,29,1.894792,0.000008,1.938399,0.000207,0.043607,xgb.XGBRegressor
2,2019-05-27 08:20:38.841277,29,1.875374,0.000077,1.919502,0.000125,0.044128,lgb.LGBMRegressor
3,2019-05-27 08:21:24.482642,74,1.891197,0.000022,1.928366,0.000156,0.037169,GradientBoostingRegressor
4,2019-05-27 08:26:08.851796,84,2.079637,0.000002,2.085509,0.000055,0.005872,RandomForestRegressor
5,2019-05-27 08:27:04.423574,74,2.108918,0.000024,2.110072,0.000028,0.001154,ExtraTreesRegressor
6,2019-05-27 08:27:08.741327,169,2.103840,0.002119,2.154801,0.008427,0.050961,Lasso
7,2019-05-27 08:27:09.585785,169,2.141680,0.002569,2.178396,0.012705,0.036715,Ridge
8,2019-05-27 08:31:12.410820,18,1.912142,0.000033,1.957761,0.000124,0.045619,RGFRegressor


In [201]:
catboost_columns = df_trial_catboost.loc[452]['param']['columns']
xgbm_columns = df_trial_xgbm.loc[1172]['param']['columns']
lgbm_columns = df_trial_lgbm.loc[2156]['param']['columns']
rf_columns = df_trial_randomforest.loc[297]['param']['columns']
extratrees_columns = df_trial_extratrees.loc[459]['param']['columns']
gradientboosting_columns = df_trial_gradientboosting.loc[306]['param']['columns']

In [202]:
pca_columns = ['5000variance_median__pca6',
 'abs_q05_6_pca1',
 '5000std_quantile75_pca4',
 "range_count{'min': 0, 'max': 1000000000000.0}_pca1"]

In [213]:
# trial_lst = [
#     df_trial_comp.loc[0], df_trial_comp.loc[1], df_trial_comp.loc[2],df_trial_comp.loc[3],
#              df_trial_comp.loc[4],df_trial_comp.loc[5],
#             df_trial_knn.loc[17], df_trial_svr.loc[7],df_trial_fm.loc[313],
#             df_trial_comp.loc[6], df_trial_comp.loc[7],
#         df_trial_mlp.loc[13], df_trial_frgf.loc[200],
#             ]

# name_lst = [
#     'cb452', 'xgbm1172', 'lgbm2156',
#     'gbm306','rf297','et459',
#     'knn17','svr7', 'fm313',
#     'lasso8','ridge15',
#     'mlp13','frgf200'
# ]


trial_lst = [
    df_trial_catboost.loc[452], df_trial_xgbm.loc[1172], df_trial_lgbm.loc[2156],df_trial_gradientboosting.loc[306],
             df_trial_randomforest.loc[297],df_trial_extratrees.loc[459],
            df_trial_knn.loc[17], df_trial_svr.loc[7],df_trial_fm.loc[313],
            df_trial_lasso.loc[8], df_trial_ridge.loc[15],
        df_trial_mlp.loc[13], df_trial_frgf.loc[200],
            ]
name_lst = [
    'cb452', 'xgbm1172', 'lgbm2156',
    'gbm306','rf297','et459',
    'knn17','svr7', 'fm313',
    'lasso8','ridge15',
    'mlp13','frgf200'
]

In [214]:
df_train_stacknet = pd.DataFrame()
df_test_stacknet = pd.DataFrame()
for df_, name_ in zip(trial_lst, name_lst):
    try:
        df_test_stacknet_i = df_['df_test_pred']
        df_test_stacknet[name_] = np.mean(df_test_stacknet_i.drop(columns=['index']).values, axis=1)
        
        df_train_stacknet_i = df_['df_valid_pred']
        df_train_stacknet[name_] = df_train_stacknet_i['predict']
        
    except Exception as e:
        print(name_, 'exception')

df_train_stacknet['index'] = df_train_stacknet_i['index']
df_train_stacknet = pd.merge(df_train_stacknet, df_train[['y','index', 'group','label']+catboost_columns+pca_columns], on='index')
df_test_stacknet['index'] = df_test_stacknet_i['index']
df_test_stacknet = pd.merge(df_test_stacknet, df_test[['index']+catboost_columns+pca_columns], on='index')

In [146]:
# df_trial_mystacknet = pd.read_pickle('../trial/mystack.pkl')

In [148]:
df_trial_mystacknet.loc[[562]][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
562,2019-05-19 20:38:58.981647,43,1.734277,0.000014,1.784253,0.000267,0.049976


In [231]:
df_trial_mystacknet.loc[562]['param']

{'columns': ['cb452',
  'xgbm1172',
  'lgbm2156',
  'gbm306',
  'rf297',
  'et459',
  'knn17',
  'svr7',
  'fm313',
  'lasso8',
  'ridge15',
  'mlp13',
  'frgf200',
  'spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'nu

In [215]:
param562 = copy.deepcopy(df_trial_mystacknet.loc[562]['param'])
param562['columns']=name_lst+catboost_columns+pca_columns
param562['scaler']['init']={}

In [216]:
df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train_stacknet, param562, df_test=df_test_stacknet, trial=mytrial, is_output_feature_importance=False, remark='redo 562')

0:	learn: 6.4873269	total: 52ms	remaining: 13.8s
1:	learn: 6.2465591	total: 104ms	remaining: 13.8s
2:	learn: 6.0178596	total: 157ms	remaining: 13.8s
3:	learn: 5.8018410	total: 207ms	remaining: 13.6s
4:	learn: 5.5953554	total: 258ms	remaining: 13.5s
5:	learn: 5.4000059	total: 310ms	remaining: 13.5s
6:	learn: 5.2122386	total: 362ms	remaining: 13.4s
7:	learn: 5.0358216	total: 412ms	remaining: 13.3s
8:	learn: 4.8693505	total: 462ms	remaining: 13.2s
9:	learn: 4.7116236	total: 512ms	remaining: 13.2s
10:	learn: 4.5615411	total: 565ms	remaining: 13.1s
11:	learn: 4.4207127	total: 616ms	remaining: 13.1s
12:	learn: 4.2885102	total: 667ms	remaining: 13s
13:	learn: 4.1638158	total: 720ms	remaining: 13s
14:	learn: 4.0469077	total: 770ms	remaining: 12.9s
15:	learn: 3.9351019	total: 822ms	remaining: 12.9s
16:	learn: 3.8344452	total: 876ms	remaining: 12.9s
17:	learn: 3.7356849	total: 926ms	remaining: 12.8s
18:	learn: 3.6428372	total: 978ms	remaining: 12.8s
19:	learn: 3.5552848	total: 1.03s	remaining: 1

161:	learn: 2.2802417	total: 8.36s	remaining: 5.42s
162:	learn: 2.2797195	total: 8.41s	remaining: 5.36s
163:	learn: 2.2793980	total: 8.46s	remaining: 5.31s
164:	learn: 2.2782366	total: 8.51s	remaining: 5.26s
165:	learn: 2.2767611	total: 8.56s	remaining: 5.21s
166:	learn: 2.2761544	total: 8.62s	remaining: 5.16s
167:	learn: 2.2755424	total: 8.67s	remaining: 5.11s
168:	learn: 2.2751855	total: 8.72s	remaining: 5.05s
169:	learn: 2.2744751	total: 8.77s	remaining: 5s
170:	learn: 2.2731470	total: 8.82s	remaining: 4.95s
171:	learn: 2.2727711	total: 8.87s	remaining: 4.9s
172:	learn: 2.2722902	total: 8.93s	remaining: 4.85s
173:	learn: 2.2713717	total: 8.98s	remaining: 4.8s
174:	learn: 2.2702919	total: 9.03s	remaining: 4.75s
175:	learn: 2.2698293	total: 9.08s	remaining: 4.69s
176:	learn: 2.2695214	total: 9.13s	remaining: 4.64s
177:	learn: 2.2690203	total: 9.18s	remaining: 4.59s
178:	learn: 2.2684192	total: 9.23s	remaining: 4.54s
179:	learn: 2.2675051	total: 9.28s	remaining: 4.49s
180:	learn: 2.266

54:	learn: 2.4445721	total: 2.85s	remaining: 11s
55:	learn: 2.4386911	total: 2.9s	remaining: 10.9s
56:	learn: 2.4326200	total: 2.95s	remaining: 10.9s
57:	learn: 2.4258292	total: 3.01s	remaining: 10.8s
58:	learn: 2.4198515	total: 3.06s	remaining: 10.8s
59:	learn: 2.4147685	total: 3.11s	remaining: 10.7s
60:	learn: 2.4100762	total: 3.16s	remaining: 10.7s
61:	learn: 2.4058089	total: 3.21s	remaining: 10.6s
62:	learn: 2.4018556	total: 3.27s	remaining: 10.6s
63:	learn: 2.3973602	total: 3.31s	remaining: 10.5s
64:	learn: 2.3940374	total: 3.37s	remaining: 10.5s
65:	learn: 2.3908452	total: 3.42s	remaining: 10.4s
66:	learn: 2.3877423	total: 3.47s	remaining: 10.4s
67:	learn: 2.3842759	total: 3.52s	remaining: 10.3s
68:	learn: 2.3808946	total: 3.57s	remaining: 10.3s
69:	learn: 2.3779648	total: 3.62s	remaining: 10.2s
70:	learn: 2.3752375	total: 3.68s	remaining: 10.2s
71:	learn: 2.3730476	total: 3.73s	remaining: 10.1s
72:	learn: 2.3701906	total: 3.78s	remaining: 10s
73:	learn: 2.3675000	total: 3.83s	re

214:	learn: 2.2457271	total: 11.1s	remaining: 2.67s
215:	learn: 2.2454663	total: 11.1s	remaining: 2.62s
216:	learn: 2.2442871	total: 11.2s	remaining: 2.57s
217:	learn: 2.2437579	total: 11.2s	remaining: 2.52s
218:	learn: 2.2434768	total: 11.3s	remaining: 2.47s
219:	learn: 2.2433297	total: 11.3s	remaining: 2.42s
220:	learn: 2.2432121	total: 11.4s	remaining: 2.36s
221:	learn: 2.2426802	total: 11.4s	remaining: 2.31s
222:	learn: 2.2421245	total: 11.5s	remaining: 2.26s
223:	learn: 2.2412781	total: 11.5s	remaining: 2.21s
224:	learn: 2.2405995	total: 11.6s	remaining: 2.16s
225:	learn: 2.2399771	total: 11.6s	remaining: 2.11s
226:	learn: 2.2396563	total: 11.7s	remaining: 2.06s
227:	learn: 2.2393934	total: 11.7s	remaining: 2s
228:	learn: 2.2386658	total: 11.8s	remaining: 1.95s
229:	learn: 2.2379250	total: 11.8s	remaining: 1.9s
230:	learn: 2.2374511	total: 11.9s	remaining: 1.85s
231:	learn: 2.2368179	total: 11.9s	remaining: 1.8s
232:	learn: 2.2362865	total: 12s	remaining: 1.75s
233:	learn: 2.23548

108:	learn: 2.3081995	total: 5.55s	remaining: 8.05s
109:	learn: 2.3069027	total: 5.6s	remaining: 8s
110:	learn: 2.3063473	total: 5.66s	remaining: 7.95s
111:	learn: 2.3045371	total: 5.71s	remaining: 7.9s
112:	learn: 2.3035331	total: 5.76s	remaining: 7.85s
113:	learn: 2.3025357	total: 5.81s	remaining: 7.8s
114:	learn: 2.3013203	total: 5.86s	remaining: 7.75s
115:	learn: 2.3007297	total: 5.92s	remaining: 7.7s
116:	learn: 2.2997972	total: 5.97s	remaining: 7.65s
117:	learn: 2.2991889	total: 6.02s	remaining: 7.6s
118:	learn: 2.2986360	total: 6.07s	remaining: 7.55s
119:	learn: 2.2979140	total: 6.12s	remaining: 7.5s
120:	learn: 2.2973247	total: 6.17s	remaining: 7.45s
121:	learn: 2.2969324	total: 6.22s	remaining: 7.4s
122:	learn: 2.2961564	total: 6.28s	remaining: 7.35s
123:	learn: 2.2952961	total: 6.33s	remaining: 7.3s
124:	learn: 2.2946205	total: 6.38s	remaining: 7.25s
125:	learn: 2.2941875	total: 6.43s	remaining: 7.2s
126:	learn: 2.2934901	total: 6.48s	remaining: 7.14s
127:	learn: 2.2927876	to

In [217]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].tail(10)

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
271,2019-05-27 07:51:48.678143,19,1.822827,0.000002,1.870929,0.000137,0.048102
272,2019-05-27 07:52:00.828852,19,1.823481,0.000002,1.871260,0.000104,0.047779
273,2019-05-27 07:52:12.967147,19,1.820711,0.000003,1.866840,0.000144,0.046129
274,2019-05-27 07:52:25.683212,19,1.820439,0.000008,1.868264,0.000090,0.047825
275,2019-05-27 08:05:48.535344,43,1.734277,0.000014,1.784253,0.000267,0.049976
276,2019-05-27 08:10:19.432898,47,1.726593,0.000017,1.777357,0.000279,0.050764
277,2019-05-27 08:15:38.443440,46,1.729906,0.000002,1.780750,0.000302,0.050844
278,2019-05-27 08:49:28.544542,47,1.728211,0.000023,1.779437,0.000236,0.051225
279,2019-05-27 08:56:56.881346,47,1.728211,0.000023,1.779437,0.000236,0.051225
280,2019-05-27 08:59:55.690163,47,1.726593,0.000017,1.777357,0.000279,0.050764


In [218]:
columns = df_trial.loc[280]['param']['columns']

In [222]:
#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.001, 100)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
                'logging_level': 'Silent'
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train_stacknet, args, df_test = df_test_stacknet, trial=mytrial, remark='tune 280')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=400)

[I 2019-05-27 09:14:14,276] Finished a trial resulted in value: 0.1263796601407585. Current best value is 0.1263796601407585 with parameters: {'num_trees': 364, 'depth': 8, 'learning_rate': 0.06069945771290027, 'l2_leaf_reg': 67.2131643134548, 'bagging_temperature': 0.7484630407511079, 'random_strength': 0.5472519316535847, 'random_state': 2149}.
[I 2019-05-27 09:15:28,850] Finished a trial resulted in value: 0.5842901373914158. Current best value is 0.1263796601407585 with parameters: {'num_trees': 364, 'depth': 8, 'learning_rate': 0.06069945771290027, 'l2_leaf_reg': 67.2131643134548, 'bagging_temperature': 0.7484630407511079, 'random_strength': 0.5472519316535847, 'random_state': 2149}.
[I 2019-05-27 09:15:52,433] Finished a trial resulted in value: 0.10236306392679763. Current best value is 0.10236306392679763 with parameters: {'num_trees': 295, 'depth': 7, 'learning_rate': 0.06538820301499242, 'l2_leaf_reg': 61.833582263733064, 'bagging_temperature': 0.9253036182056417, 'random_str

[I 2019-05-27 09:44:36,681] Finished a trial resulted in value: 0.3153287806047675. Current best value is 0.009915481573466842 with parameters: {'num_trees': 220, 'depth': 4, 'learning_rate': 0.012491808316224411, 'l2_leaf_reg': 3.714812577982709, 'bagging_temperature': 0.8052072435608288, 'random_strength': 0.6993543894622959, 'random_state': 7362}.
[I 2019-05-27 09:44:50,174] Finished a trial resulted in value: 0.16730351940834173. Current best value is 0.009915481573466842 with parameters: {'num_trees': 220, 'depth': 4, 'learning_rate': 0.012491808316224411, 'l2_leaf_reg': 3.714812577982709, 'bagging_temperature': 0.8052072435608288, 'random_strength': 0.6993543894622959, 'random_state': 7362}.
[I 2019-05-27 09:45:02,577] Finished a trial resulted in value: 0.32365844386562. Current best value is 0.009915481573466842 with parameters: {'num_trees': 220, 'depth': 4, 'learning_rate': 0.012491808316224411, 'l2_leaf_reg': 3.714812577982709, 'bagging_temperature': 0.8052072435608288, 'ran

[I 2019-05-27 10:00:00,780] Finished a trial resulted in value: 0.0702336825178799. Current best value is 0.007537201799727981 with parameters: {'num_trees': 284, 'depth': 2, 'learning_rate': 0.01150718749356178, 'l2_leaf_reg': 26.237671500567213, 'bagging_temperature': 0.7519486600645952, 'random_strength': 0.9656645598835933, 'random_state': 9034}.
[I 2019-05-27 10:00:17,878] Finished a trial resulted in value: 0.2929096987401897. Current best value is 0.007537201799727981 with parameters: {'num_trees': 284, 'depth': 2, 'learning_rate': 0.01150718749356178, 'l2_leaf_reg': 26.237671500567213, 'bagging_temperature': 0.7519486600645952, 'random_strength': 0.9656645598835933, 'random_state': 9034}.
[I 2019-05-27 10:00:24,243] Finished a trial resulted in value: 0.11828263243407651. Current best value is 0.007537201799727981 with parameters: {'num_trees': 284, 'depth': 2, 'learning_rate': 0.01150718749356178, 'l2_leaf_reg': 26.237671500567213, 'bagging_temperature': 0.7519486600645952, 'r

[I 2019-05-27 10:12:34,847] Finished a trial resulted in value: 0.040085825687723925. Current best value is 0.007537201799727981 with parameters: {'num_trees': 284, 'depth': 2, 'learning_rate': 0.01150718749356178, 'l2_leaf_reg': 26.237671500567213, 'bagging_temperature': 0.7519486600645952, 'random_strength': 0.9656645598835933, 'random_state': 9034}.
[I 2019-05-27 10:12:42,542] Finished a trial resulted in value: 0.16834680834277313. Current best value is 0.007537201799727981 with parameters: {'num_trees': 284, 'depth': 2, 'learning_rate': 0.01150718749356178, 'l2_leaf_reg': 26.237671500567213, 'bagging_temperature': 0.7519486600645952, 'random_strength': 0.9656645598835933, 'random_state': 9034}.
[I 2019-05-27 10:12:51,072] Finished a trial resulted in value: 0.12905667207982116. Current best value is 0.007537201799727981 with parameters: {'num_trees': 284, 'depth': 2, 'learning_rate': 0.01150718749356178, 'l2_leaf_reg': 26.237671500567213, 'bagging_temperature': 0.7519486600645952,

[I 2019-05-27 10:24:48,419] Finished a trial resulted in value: 0.31834954021731554. Current best value is 0.0070531665839572545 with parameters: {'num_trees': 250, 'depth': 2, 'learning_rate': 0.01034894052894668, 'l2_leaf_reg': 8.009703472210383, 'bagging_temperature': 0.6889231926903113, 'random_strength': 0.9929800591145785, 'random_state': 7400}.
[I 2019-05-27 10:25:26,223] Finished a trial resulted in value: 0.18120533712750986. Current best value is 0.0070531665839572545 with parameters: {'num_trees': 250, 'depth': 2, 'learning_rate': 0.01034894052894668, 'l2_leaf_reg': 8.009703472210383, 'bagging_temperature': 0.6889231926903113, 'random_strength': 0.9929800591145785, 'random_state': 7400}.
[I 2019-05-27 10:26:00,704] Finished a trial resulted in value: 0.409278747345089. Current best value is 0.0070531665839572545 with parameters: {'num_trees': 250, 'depth': 2, 'learning_rate': 0.01034894052894668, 'l2_leaf_reg': 8.009703472210383, 'bagging_temperature': 0.6889231926903113, 'r

[I 2019-05-27 10:45:27,592] Finished a trial resulted in value: 0.3621107127425449. Current best value is 0.0070531665839572545 with parameters: {'num_trees': 250, 'depth': 2, 'learning_rate': 0.01034894052894668, 'l2_leaf_reg': 8.009703472210383, 'bagging_temperature': 0.6889231926903113, 'random_strength': 0.9929800591145785, 'random_state': 7400}.
[I 2019-05-27 10:45:35,550] Finished a trial resulted in value: 0.0625203648437289. Current best value is 0.0070531665839572545 with parameters: {'num_trees': 250, 'depth': 2, 'learning_rate': 0.01034894052894668, 'l2_leaf_reg': 8.009703472210383, 'bagging_temperature': 0.6889231926903113, 'random_strength': 0.9929800591145785, 'random_state': 7400}.
[I 2019-05-27 10:45:41,777] Finished a trial resulted in value: 0.03221690482030104. Current best value is 0.0070531665839572545 with parameters: {'num_trees': 250, 'depth': 2, 'learning_rate': 0.01034894052894668, 'l2_leaf_reg': 8.009703472210383, 'bagging_temperature': 0.6889231926903113, 'r

[I 2019-05-27 10:59:05,797] Finished a trial resulted in value: 0.06183568608794956. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542, 'random_strength': 0.9023782701971889, 'random_state': 8892}.
[I 2019-05-27 10:59:10,588] Finished a trial resulted in value: 0.024835841650271945. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542, 'random_strength': 0.9023782701971889, 'random_state': 8892}.
[I 2019-05-27 10:59:15,731] Finished a trial resulted in value: 0.006603728964373607. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576

[I 2019-05-27 11:04:09,761] Finished a trial resulted in value: 0.0119499662378095. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542, 'random_strength': 0.9023782701971889, 'random_state': 8892}.
[I 2019-05-27 11:04:21,470] Finished a trial resulted in value: 0.06988484121351815. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542, 'random_strength': 0.9023782701971889, 'random_state': 8892}.
[I 2019-05-27 11:04:34,581] Finished a trial resulted in value: 0.04053238711579313. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542

[I 2019-05-27 11:16:31,445] Finished a trial resulted in value: 0.0636014274932932. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542, 'random_strength': 0.9023782701971889, 'random_state': 8892}.
[I 2019-05-27 11:16:35,872] Finished a trial resulted in value: 0.0652655364310294. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542, 'random_strength': 0.9023782701971889, 'random_state': 8892}.
[I 2019-05-27 11:16:42,343] Finished a trial resulted in value: 0.032826869869848285. Current best value is 0.005045020514163365 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.012174078439872504, 'l2_leaf_reg': 23.108269982452747, 'bagging_temperature': 0.6948723551576542

In [228]:
df_trial = pd.DataFrame(mytrial)
df_trial[(df_trial['remark']=='tune 280')&(df_trial['mae_diff']<.052)][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae'])

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
469,2019-05-27 10:24:12.606868,47,1.730152,1.548608e-05,1.778465,2.491730e-04,0.048313
501,2019-05-27 10:41:11.685309,47,1.728984,6.661831e-05,1.779580,2.160474e-04,0.050596
602,2019-05-27 11:03:03.795910,47,1.728197,1.042951e-05,1.779687,2.724670e-04,0.051490
352,2019-05-27 09:53:57.091834,47,1.728598,2.004541e-05,1.780173,2.099344e-04,0.051575
527,2019-05-27 10:47:20.402179,47,1.731551,7.500784e-06,1.780475,2.883831e-04,0.048924
401,2019-05-27 10:06:42.108002,47,1.730170,1.226343e-05,1.781682,3.152299e-04,0.051512
356,2019-05-27 09:54:22.433478,47,1.734247,2.660389e-05,1.782001,2.969209e-04,0.047753
672,2019-05-27 11:19:17.875451,47,1.732810,1.839120e-05,1.782096,2.268439e-04,0.049285
643,2019-05-27 11:08:21.319790,47,1.737528,5.835197e-07,1.782444,3.358070e-04,0.044916
385,2019-05-27 10:01:00.373780,47,1.736523,2.452969e-06,1.782964,2.621412e-04,0.046440


In [232]:
idx=469
df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_mystacknet_pca_{}.csv'.format(idx), index=False)

In [229]:
df_trial.loc[469]['param']

{'columns': ['cb452',
  'xgbm1172',
  'lgbm2156',
  'gbm306',
  'rf297',
  'et459',
  'knn17',
  'svr7',
  'fm313',
  'lasso8',
  'ridge15',
  'mlp13',
  'frgf200',
  'spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05',
  '5000variance_median__pca6',
  'abs_q05_6_pca1',
  '5000std_quantile75_pca4',
  "range_count{'min': 0, 'max': 1000000000000.0}_pca1"],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shu

In [230]:
df_trial.loc[280]['param']

{'columns': ['cb452',
  'xgbm1172',
  'lgbm2156',
  'gbm306',
  'rf297',
  'et459',
  'knn17',
  'svr7',
  'fm313',
  'lasso8',
  'ridge15',
  'mlp13',
  'frgf200',
  'spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05',
  '5000variance_median__pca6',
  'abs_q05_6_pca1',
  '5000std_quantile75_pca4',
  "range_count{'min': 0, 'max': 1000000000000.0}_pca1"],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shu

In [233]:
df_trial.to_pickle('../trial/stacknet_pca.pkl')